In [4]:
from pyspark.sql import SparkSession, Row
import __credential__

# include spark-xml package
from os import environ
environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.10:0.4.1 pyspark-shell' 
environ['PYSPARK_PYTHON']='/home/ubuntu/anaconda3/bin/python'
environ['PYSPARK_DRIVER_PYTHON']='/home/ubuntu/anaconda3/bin/jupyter'

In [5]:
spark = SparkSession \
    .builder \
    .master(__credential__.spark_host) \
    .appName("xml_reader") \
    .getOrCreate()
    

In [6]:
# Load clinical.xml file from Amazon S3
xml_schema = spark.read.format('com.databricks.spark.xml') \
    .options(rowTag='brca:patient') \
    .load(__credential__.module_xml_reader_testfile)
print(type(xml_schema))
xml_schema.printSchema()

xml_schema_rdd = xml_schema.rdd.map(lambda x:Row( \
                    stage = x['shared_stage:stage_event']['shared_stage:pathologic_stage']._VALUE, \
                    primary_site = x['clin_shared:tumor_tissue_site']._VALUE, \
                    gender = x['shared:gender']._VALUE))
xml_schema_rdd.take(3)

<class 'pyspark.sql.dataframe.DataFrame'>
root
 |-- admin:additional_studies: string (nullable = true)
 |-- brca:anatomic_neoplasm_subdivisions: struct (nullable = true)
 |    |-- clin_shared:anatomic_neoplasm_subdivision: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _cde: long (nullable = true)
 |    |    |-- _cde_ver: double (nullable = true)
 |    |    |-- _display_order: long (nullable = true)
 |    |    |-- _owner: string (nullable = true)
 |    |    |-- _preferred_name: string (nullable = true)
 |    |    |-- _procurement_status: string (nullable = true)
 |    |    |-- _restricted: boolean (nullable = true)
 |    |    |-- _source_system_identifier: long (nullable = true)
 |    |    |-- _tier: long (nullable = true)
 |    |    |-- _xsd_ver: double (nullable = true)
 |-- brca:axillary_lymph_node_stage_method_type: struct (nullable = true)
 |    |-- _VALUE: string (nullable = true)
 |    |-- _cde: long (nullable = true)
 |    |-- _cde_ver: 

[Row(gender='FEMALE', primary_site='Breast', stage='Stage IIA')]